In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
from xgboost import XGBClassifier

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
dataset = load_dataset("paws", "labeled_final")
train_sentences=[]
#49401
for i in range(0,49401):
    sent1=dataset["train"][i]["sentence1"]
    sent2=dataset["train"][i]["sentence2"]
    label=dataset["train"][i]["label"]
    tu=(sent1,sent2,label)
    train_sentences.append(tu)
test_sentences=[]
for i in range(0,8000):
    sent1=dataset["test"][i]["sentence1"]
    sent2=dataset["test"][i]["sentence2"]
    label=dataset["test"][i]["label"]
    tu=(sent1,sent2,label)
    test_sentences.append(tu)

In [4]:
train_sim=[]
train_label=[]

for sent1, sent2, label in tqdm(train_sentences):
    embedding1 = model.encode(sent1)
    embedding2 = model.encode(sent2)
    embedding1_np = np.array(embedding1).reshape(1, -1)
    embedding2_np = np.array(embedding2).reshape(1, -1)
    similarity = cosine_similarity(embedding1_np, embedding2_np)
    train_sim.append(similarity)
    train_label.append(label)

# Convert lists to arrays
train_similarity = np.array(train_sim).reshape(-1, 1)
train_labels = np.array(train_label)

100%|██████████| 49401/49401 [20:20<00:00, 40.48it/s] 


In [5]:
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(train_similarity, train_labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [7]:
test_similarity = []
true_labels = []
for phrase1, phrase2, true_label in tqdm(test_sentences):
    embedding1 = model.encode(phrase1)
    embedding2 = model.encode(phrase2)
    embedding1_np = np.array(embedding1).reshape(1, -1)
    embedding2_np = np.array(embedding2).reshape(1, -1)
    similarity = cosine_similarity(embedding1_np, embedding2_np)  # Calculate similarity
    test_similarity.append(similarity)
    true_labels.append(true_label)

# Convert lists to arrays
test_similarity = np.array(test_similarity)

# Evaluate performance
predicted_labels = xgb_classifier.predict(test_similarity.reshape(-1, 1))
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

100%|██████████| 8000/8000 [03:13<00:00, 41.29it/s]

Accuracy: 0.613125
